# CMA-ES


**Disclaimer:** We make use of the implementation available at [PyPi](https://pypi.org/project/cma/) <cite data-cite="pycma"></cite> published by the author Nikolaus Hansen under the BSD license. 


CMA-ES which was proposed in <cite data-cite="cmaes"></cite>. Moreover, a comparing review can be found in <cite data-cite="cmaes-review"></cite>. 
CMA-ES stands for covariance matrix adaptation evolution strategy. Evolution strategies (ES) are stochastic, derivative-free methods for numerical optimization of non-linear or non-convex continuous optimization problems. They belong to the class of evolutionary algorithms and evolutionary computation. An evolutionary algorithm is broadly based on the principle of biological evolution, namely the repeated interplay of variation (via recombination and mutation) and selection: in each generation (iteration) new individuals (candidate solutions) are generated by variation, usually in a stochastic way, of the current parental individuals. Then, some individuals are selected to become the parents in the next generation based on their fitness or objective function value 
$f(x)$. Like this, over the generation sequence, individuals with better and better $f$-values are generated.
(excerpt from [Wikipedia](https://en.wikipedia.org/wiki/CMA-ES)).

### Example

In [1]:
import numpy as np

from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.factory import get_problem
from pymoo.optimize import minimize

problem = get_problem("sphere")

algorithm = CMAES(x0=np.random.random(problem.n_var))

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print(f"Best solution found: \nX = {res.X}\nF = {res.F}\nCV= {res.CV}")


Best solution found: 
X = [0.50000004 0.49999999 0.49999997 0.49999997 0.49999999 0.50000002
 0.49999995 0.49999999 0.49999999 0.49999998]
F = [7.65534518e-15]
CV= [0.]


CMA-ES already has several stopping criteria implemented. However, as for other algorithms, the number of iterations or function evaluations can be directly passed to `minimize`.

In [2]:
res = minimize(problem,
               algorithm,
               ('n_iter', 10),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen |  n_eval |     fopt     |    sigma     | min std  | max std  |   axis  
    1 |       1 |  0.604499915 |  0.100000000 |  0.10000 |  0.10000 |  1.00005
    2 |      11 |  0.532900591 |  0.095801004 |  0.09270 |  0.09741 |  1.16332
    3 |      21 |  0.451474125 |  0.098357677 |  0.09295 |  0.10247 |  1.28202
    4 |      31 |  0.333742978 |  0.101774490 |  0.09489 |  0.11094 |  1.39779
    5 |      41 |  0.247022231 |  0.104440269 |  0.09595 |  0.11372 |  1.41370
    6 |      51 |  0.136045325 |  0.119075984 |  0.10443 |  0.13672 |  1.61850
    7 |      61 |  0.128434852 |  0.128598196 |  0.11246 |  0.15371 |  1.69380
    8 |      71 |  0.104217515 |  0.127648197 |  0.11120 |  0.15345 |  1.76726
    9 |      81 |  0.104217515 |  0.126210993 |  0.10895 |  0.15020 |  1.84915
   10 |      91 |  0.096641551 |  0.119587279 |  0.10359 |  0.14205 |  1.86471
Best solution found: 
X = [0.34257037 0.62200624 0.49244253 0.3630729  0.52048598 0.49023415
 0.53078558 0.51720187 0.46296191 0.31

In [3]:
res = minimize(problem,
               algorithm,
               ('n_evals', 50),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen |  n_eval |     fopt     |    sigma     | min std  | max std  |   axis  
    1 |       1 |  0.604499915 |  0.100000000 |  0.10000 |  0.10000 |  1.00005
    2 |      11 |  0.485417468 |  0.098428231 |  0.09271 |  0.10585 |  1.23936
    3 |      21 |  0.398263548 |  0.106768322 |  0.09876 |  0.11880 |  1.35923
    4 |      31 |  0.282700042 |  0.124959146 |  0.11306 |  0.13878 |  1.47296
    5 |      41 |  0.282283633 |  0.125425471 |  0.11223 |  0.13792 |  1.50595
    6 |      51 |  0.168176465 |  0.127535649 |  0.11465 |  0.14277 |  1.53310
Best solution found: 
X = [0.62693251 0.48362336 0.38497379 0.34059669 0.56831997 0.23592506
 0.62988836 0.50021669 0.37960744 0.5859498 ]
F = [0.16817647]


Also, easily restarts can be used, which are known to work very well on multi-modal functions. For instance, `Rastrigin` can be solved rather quickly by:

In [4]:
problem = get_problem("rastrigin")

algorithm = CMAES(restarts=10, restart_from_best=True)

res = minimize(problem,
               algorithm,
               ('n_evals', 2500),
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [ 9.94958644e-01 -5.07348208e-09]
F = [0.99495906]


Our framework internally calls the `cma.fmin2` function. All parameters that can be used there either as a keyword argument or an option can also be passed to the `CMAES` constructor.
An example with a few selected `cma.fmin2` parameters is shown below:

In [5]:
import numpy as np
from pymoo.util.normalization import denormalize

# define an intitial point for the search
np.random.seed(1)
x0 = denormalize(np.random.random(problem.n_var), problem.xl, problem.xu)

algorithm = CMAES(x0=x0,
                 sigma=0.5,
                 restarts=2,
                 maxfevals=np.inf,
                 tolfun=1e-6,
                 tolx=1e-6,
                 restart_from_best=True,
                 bipop=True)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [-5.88141386e-07  9.94958218e-01]
F = [0.99495906]


For more details about hyperparameters, we refer to the software documentation of the `fmin2` in CMA-ES, which can be found [here](http://cma.gforge.inria.fr/apidocs-pycma/cma.evolution_strategy.html#fmin2).
A quick explanation of possible parameters is also provided in the API documentation below.

### API